# Bencmark d'un modèle simple (drag and drop) sur Azure ML studio (classic)

Voici à quoi ressemble le modèle final :

<img src="./img/studio_classic_web_service.jpg" style="margin-left:auto; margin-right:auto; display: block; height: 800px">

Voici le dérouler de cette pipeline :

- On retrouve en haut le module d'import des données qui importe un échantillion de 10000 données parfaitement stratifié ( 5000 bad tweet & 5000 good tweet) & "l'input" de notre web service (qui se base sur les colonnes selectionné à l'étape suivante)
- Après cette étape j'ai réaliser une selection des colonnes pour ne prendre que la colonne de "text" et de "sentiments"
- J'ai ensuite réaliser un preprocessed du text simple qui nettoie la column "text" 
- Après ça j'ai récupérer le jeu de données du vocabulaire TF-IDF que j'ai réaliser durant la phase d'entrainement du modèle que j'ai du coup utiliser pour générer ma matrice TF-IDF
- et pour finir on retrouve le modèle final entrainer qui permet la prédiction de sentiment du tweet

In [1]:
import urllib.request
import json 
import os

api_key = os.environ["ML_STUDIO_CLASSIQUE_KEY"]

url = os.environ["ML_STUDIO_CLASSIQUE_ENDPOINT"]
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

In [3]:
def prepare_body(value : str):
    data =  {
        "Inputs": {
            "text":{
                "ColumnNames": ["text"],
                "Values": [[value]]
                },
            },
        "GlobalParameters": {}
        }
    return str.encode(json.dumps(data))
    
def prepare_response(resp,dic=None):
    json_result = json.loads(resp)
    values = json_result["Results"]["score"]["value"]["Values"]
    if dic is None:
        dic = {
            "bad_score":[],
            "good_score":[],
            "prediction":[]
        }
    for v in values:
        bad_res = v[0]
        good_res = v[1]
        predict = v[2]
        dic["bad_score"].append(bad_res)
        dic["good_score"].append(good_res)
        dic["prediction"].append(predict)
    return dic

def send_request(list_value : list):
    result = None
    for value in list_value:
        body = prepare_body(value)
        req = urllib.request.Request(url, body, headers) 
        try:
            response = urllib.request.urlopen(req)
            resp = response.read()
            if result is None:
                result = prepare_response(resp)
            else:
                result = prepare_response(resp,dic=result)
        except:
            print("The request failed value :",value)  
    return result

In [4]:
list_value = ["I feel like i have a hangover... But i didn't really drink alot last night.",
              "I love banana !","i lose followers all the time.  haha","I LOVE @Health4UandPets u guys r the best!!"]
result = send_request(list_value)

In [5]:
print(result)

{'bad_score': ['0.84832626581192', '0.280947744846344', '0.339496046304703', '0.349292606115341'], 'good_score': ['0.15167373418808', '0.719052255153656', '0.66050398349762', '0.650707423686981'], 'prediction': ['bad', 'good', 'good', 'good']}


## Benchmark azure studio

In [6]:
import pandas as pd
test_sample = pd.read_csv("./data/sample_test_dataset.csv",index_col=0)
print("Dimensionnalité :",test_sample.shape)
test_sample.head()

Dimensionnalité : (4000, 2)


,text,sentiment
0,Chillin @ the beach with my girl brit-brit wat...,good
1,@indraherlambang really wanna say that ure lik...,good
2,"@ambermac Gotta love Brad Pitt's performance, ...",good
3,Just realized that Matthew Sweet &amp; Susanna...,good
4,"@marksphone sorry, meant that to come from my ...",good


In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

def benchmark(y_pred, y_pred_prob, y_true):
    print("accuracy score :",accuracy_score(y_true,y_pred))
    print("AUC score :",roc_auc_score(y_true,y_pred_prob))
    cm = confusion_matrix(y_true,y_pred)
    print(classification_report(y_true,y_pred))
    return pd.DataFrame(cm,columns=["good","bad"],index=["good","bad"])

In [8]:
document_send = test_sample["text"].tolist()
y_true = test_sample["sentiment"].tolist()

In [12]:
%%time
result = send_request(document_send)

CPU times: user 10.6 s, sys: 3.45 s, total: 14 s
Wall time: 55min 43s


In [13]:
y_pred = result["prediction"]
y_pred_proba = result["good_score"]

In [16]:
benchmark(y_pred,y_pred_proba,y_true)

accuracy score : 0.654
AUC score : 0.71521175
              precision    recall  f1-score   support

         bad       0.69      0.55      0.61      2000
        good       0.63      0.76      0.69      2000

    accuracy                           0.65      4000
   macro avg       0.66      0.65      0.65      4000
weighted avg       0.66      0.65      0.65      4000



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


,good,bad
good,1101,899
bad,485,1515


Après environ 1H de requête à l'API studio ML (classic) on obtiens un score de 0.654 d'accuracy et 0.71 d'AUC qui est assez faible, on vois cependant que l'API classe correctement majoritairement les Bad tweet. 

<img src="./img/result_metric_studio_classic_test.jpg" style="margin-left:auto; margin-right:auto; display: block; height: 500px">